In [4]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession

In [5]:
sc = SparkContext(master='local', appName='TransformacionesYacciones')

your 131072x1 screen size is bogus. expect trouble


23/08/28 13:37:13 WARN Utils: Your hostname, HP resolves to a loopback address: 127.0.1.1; using 172.28.10.209 instead (on interface eth0)
23/08/28 13:37:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/28 13:37:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark = SparkSession(sc)

In [46]:
spark

# RDD

In [48]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

Solo ejecutar `collect()` cuando la cantidad de datos que queremos mostrar es muy pequeña
Ya que cabe la posibilidad de que en nuestros datos existan miles de millones de datos
Y esto es muy **peligroso** en produccion ya que puede tumbar el servidor o puede tar varios minutos
en ejecutarse atrasando toda la produccion

In [8]:
rdd1.collect()

[1, 2, 3]

Cargamos un archivo a un RDD es comun ver que utilizen una lambda

In [11]:
path = '/home/armando/sparkCurso/data/'

equipos_olimpicosRDD = sc.textFile(path+'paises.csv') \
    .map(lambda line: line.split(','))

En ligar de `collect()` usaremos `take()` que solo muestra una cantidad determinada de valores

In [12]:
equipos_olimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

La siguiente Query toma los equipos olimpicos toma el parametro de la siglas para al ultimo hacerle metodos de SQL

como lo son el `distinct` que solo toma valores unicos 

y `count` que hace un conteo de todos los resultados unicos

El header es importante que lo toma desde el 0
|id|equipo  |sigla|
|--|--------|-----|
| 0|1       |2    |

In [13]:
equipos_olimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

Esta Query lo que hace es que toma una llave`(x[2])` en este caso es la sigla del pais y un valor`(x[1])`
que es el equipo

- `groupByKey` los agrupara con la llave que es la sigla
- `mapValues(len)` recorera todos los valores y el parametro len retornara cuantos equipos hay en cada pais


In [14]:
equipos_olimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

- `filter` filtra nuestra data de acuerdo a un parametro en este caso obtenemos los equipo de **MEX**

In [15]:
eqipos_MEX = equipos_olimpicosRDD.filter(lambda l: 'MEX' in l)
eqipos_MEX.collect()

[['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['193', 'Chamukina', 'MEX'],
 ['656', 'Mexico', 'MEX'],
 ['657', 'Mexico-1', 'MEX'],
 ['658', 'Mexico-2', 'MEX'],
 ['700', 'Nausikaa 4', 'MEX'],
 ['1053', 'Tlaloc', 'MEX'],
 ['1168', 'Xolotl', 'MEX']]

Para unir data tenemos el metodo de `union` que solo toma como parametro el RDD que queremos indexar

In [16]:
deportista_olimpicoRDD = sc.textFile(path+'deportista.csv', ) \
    .map(lambda l: l.split(','))

deportista_olimpicoRDD2 = sc.textFile(path+'deportista2.csv', ) \
    .map(lambda l: l.split(','))
    
deportista_olimpicoRDD = deportista_olimpicoRDD.union(deportista_olimpicoRDD2)

In [17]:
deportista_olimpicoRDD.count()

135572

Para unir dos RDDs de archivos diferentes tiene que tener una relacion entre ellos una ForeingKey
<image src="data/modelo_relacional.jpg" alt="Descripción de la imagen">

In [18]:
equipos_olimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [19]:
deportista_olimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

Vemos que la FK es el id del equipo 

De igual manera que tomaremos la ultima columna del deportista junto con todos los valores que este tiene menos el ultimo

En el join juntaremos solo el id del pais y las siglas del mismo

`takeSample` toma 3 parametros 
- El primero es que si queremos que los valores se repitan
- El segundo la cantidad de ejemplos que queremos mostrar
- Y el tercero es una semilla que le asignamos

In [20]:
equipo_deportista = equipos_olimpicosRDD.join(deportista_olimpicoRDD).take(6)

In [21]:
deportista_paises = deportista_olimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
    .join(equipos_olimpicosRDD.map(lambda x: [x[0], x[2]]))

deportista_paises.take(6)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN'))]

Obtenemos el archivo de resultado

In [22]:
resultadoRDD = sc.textFile(path+'resultados.csv', ) \
    .map(lambda l: l.split(','))

Filtramos omitiendo a todos aquellos que no han ganado una medalla

In [23]:
resultado_ganadorRDD = resultadoRDD.filter(lambda l: 'NA' not in l[1])

In [24]:
resultado_ganadorRDD.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [25]:
paises_Medallas = deportista_paises.join(resultado_ganadorRDD)
paises_Medallas.take(6)

[('1090',
  ((['9150', 'Yelena Yuryevna Bechke Petrova Ellis ', '2', '26', '158', '48'],
    'EUN'),
   'Bronze')),
 ('1090',
  ((['9819', 'Gennady Vladimirovich Belyakov', '1', '23', '171', '84'], 'EUN'),
   'Bronze')),
 ('1090',
  ((['41896', 'Andrey Vladimirovich Gorokhov', '1', '23', '185', '92'], 'EUN'),
   'Bronze')),
 ('1090',
  ((['64345', 'Sergey Valeryevich Kruglov', '1', '31', '0', '0'], 'EUN'),
   'Bronze')),
 ('1090',
  ((['70906', 'Igor Vladimirovich Lobanov', '1', '22', '181', '78'], 'EUN'),
   'Bronze')),
 ('1090',
  ((['90775', 'Irina Vladimirovna Palina', '2', '22', '162', '60'], 'EUN'),
   'Bronze'))]

In [26]:
valores_medallas = {
    'Gold': 7,
    'Silver': 5,
    'Bronze': 4
}

Ahora se sumara el puntaje de cada pais de acuerdo cuantas medallas recolecto

Teniendo en cuenta que
- `x[1][0][-1]` accede al pais
- `x[1][1]` accede a las medallas

In [27]:
# ('1090',((['9150', 'Yelena Yuryevna Bechke Petrova Ellis ', '2', '26', '158', '48'],'EUN'),'Bronze')),
paises_Medallas.map(lambda x: [x[1][0][-1], x[1][1]]).take(4)

[['EUN', 'Bronze'], ['EUN', 'Bronze'], ['EUN', 'Bronze'], ['EUN', 'Bronze']]

Sabiendo esto podemos tomar el diccionario **valores_medallas** y acceder a un valor numerico y no a una cadena de texto

In [28]:
paises_Medallas = paises_Medallas\
    .map(lambda x: (x[1][0][-1], valores_medallas[x[1][1]]))

paises_Medallas.take(4)

[('EUN', 4), ('EUN', 4), ('EUN', 4), ('EUN', 4)]

In [29]:
from operator import add

conclusion = paises_Medallas.reduceByKey((add)) \
    .sortBy(lambda x: x[1], ascending = False)

conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

# DataFrame
Ahora ver el archivo [3_DataFrames.ipynb](3_DataFrames.ipynb) para tener contexto de como o porque usarlos

In [30]:
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.types import Row

In [31]:
sqlContext = SQLContext(sc)

/home/armando/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# RDD a DataFrame
Se puede convertir un RDD a un DataFrame, pero para eso debemos de hacer una serie de pasos:
- Si es que tiene encabezado eleminarlo
- Convertir cada columna en su tipo de dato correspondiente
- Definir un schema en el cual especificamos el tipo de dato de cada columna
- Crear el DataFrame con un SQLContext

## Ejemplo 1

In [32]:
def eleminar_encabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [33]:
deportista_olimpicoRDD = deportista_olimpicoRDD \
    .mapPartitionsWithIndex(eleminar_encabezado)

In [49]:
deportista_olimpicoRDD = deportista_olimpicoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
))

In [35]:
schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False)
])

In [36]:
deportista_olimpicoDF = sqlContext.createDataFrame(deportista_olimpicoRDD, schema)

In [37]:
deportista_olimpicoDF.show(6)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
|            6|     Per Knut Aaland|     1|  31|   188|75.0|     1096|
+-------------+--------------------+------+----+------+----+---------+
only showing top 6 rows



## Ejemplo 2

In [38]:
resultado_ganadorRDD = resultado_ganadorRDD \
    .mapPartitionsWithIndex(eleminar_encabezado)

In [39]:
resultado_ganadorRDD = resultado_ganadorRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4])
))

In [40]:
schema = StructType([
    StructField('resultado_id', IntegerType(), False),
    StructField('medalla', StringType(), False),
    StructField('deportista_id', IntegerType(), False),
    StructField('juego_id', IntegerType(), False),
    StructField('evento_id', IntegerType(), False)
])

In [41]:
sqlContext.createDataFrame(resultado_ganadorRDD, schema).show(6)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           4|   Gold|            4|       2|        4|
|          38| Bronze|           15|       7|       19|
|          39| Bronze|           15|       7|       20|
|          41| Bronze|           16|      50|       14|
|          42| Bronze|           17|      17|       21|
|          43|   Gold|           17|      17|       22|
+------------+-------+-------------+--------+---------+
only showing top 6 rows



# DataFrame directo de un archivo
En este caso solo omitimos el paso de quitarle los encabezados
- Definir un schema con los tipos de datos
- Crear DataFrame de igual forma con SQLContext
  - Definir que tipo de archivo vamos a leer
  - Si es que el archivo tiene encabezados darle en True
  - Asignarle el schema
  - Cargar la ruta del archivo

In [43]:
schema = StructType([
    StructField("evento_id", IntegerType(), False),
    StructField("evento", StringType(), False),
    StructField("deporte_id", IntegerType(), False)
])

In [44]:
eventoDF = sqlContext.read.format('csv') \
    .option('header', True) \
    .schema(schema) \
    .load(path+'evento.csv')

eventoDF.show(6)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
|        6|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 6 rows



In [50]:
schema = StructType([
    StructField("deporte_id", IntegerType(), False),
    StructField("deporte", StringType(), False),
])

In [51]:
eventoDF = sqlContext.read.format('csv') \
    .option('header', True) \
    .schema(schema) \
    .load(path+'deporte.csv')

eventoDF.show(6)

+----------+--------------------+
|deporte_id|             deporte|
+----------+--------------------+
|         1|          Basketball|
|         2|                Judo|
|         3|            Football|
|         4|          Tug-Of-War|
|         5|       Speed Skating|
|         6|Cross Country Skiing|
+----------+--------------------+
only showing top 6 rows



In [55]:
def createDF(nombre, schema):
    df = sqlContext.read.format('csv') \
        .option('header', True) \
        .schema(schema) \
        .load(path+nombre)
    df.show(5)
    
    return df

In [56]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("equipo", StringType(), True),
    StructField("sigla", StringType(), True)
])

paisesDF = createDF('paises.csv', schema)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



In [57]:
sc.stop()
spark.stop()